# Optimization Problem

The IAA Automobile company produces four types of cars (subcompact, compact, intermediate and luxury) and trucks and vans.  Vendor capacities limit the total production (on all cars, trucks and vans) to 1.2 million vehicles per year.  



Subcompacts and compacts are built together in a facility with total capacity of 620,000.

Intermediate and luxury cars are built together in a facility with total capacity of 400,000.

Trucks and vans are built together in a facility with total capacity of 275,000.

IAA's marketing strategy requires that subcompacts and compacts constitute AT LEAST HALF of the four car types (note that this is for CARS only!!).

The corporate Average Fuel Economy standards require an average fuel economy of at least 27 mpg.

The goal is to maximize the profit.

Additional information is shown in table below:

| Type | Profit ($/vehicle) | Market Potential | Fuel Economy (mpg) |
| --- | --- | --- | --- |
| subcompact | 150 | 600 | 40 |
| compact | 225 | 400 | 34 |
| intermediate | 250 | 300 | 15 |
| luxury | 500 | 225 | 12 |
| truck | 400 | 325 | 20 |
| van | 200 | 100 | 25 |

In [1]:
# import package
from gurobipy import *

Write out the objective function using the following decision variables:
- S = subcompact
- C = compact 
- I = intermediate
- L = luxury
- T = truck
- V = van

Maximize: $Y = 150S + 225C + 250I + 500L + 400T + 200V$

***

Total Annual Capacity constraint:

$S + C + I + L + T + V <= 1200000$

***

Subcompacts and compacts total capacity constraint:

$S + C <= 620000$

***

Corporate average fuel economy constraint:

$13S + 7C - 12I - 15L - 7T - 2V >= 0 $

In [3]:
# create model object
m = Model('blending model')

# create variables
s = m.addVar(vtype=GRB.CONTINUOUS, name="Subcompact")
c = m.addVar(vtype=GRB.CONTINUOUS, name="Compact")
i = m.addVar(vtype=GRB.CONTINUOUS, name="intermediate")
l = m.addVar(vtype=GRB.CONTINUOUS, name="Luxury")
t = m.addVar(vtype=GRB.CONTINUOUS, name="Truck")
v = m.addVar(vtype=GRB.CONTINUOUS, name="Van")

# set objective
m.setObjective(150*s  + 225*c + 250*i + 500*l + 400*t + 200*v, GRB.MAXIMIZE)

""" Add constraints including implicit"""

# capacity constraints
m.addConstr(s + c+ i + l + t + v <= 1200000, name='Annual capacity')
m.addConstr(s + c  <= 620000, name='subcompact compact')
m.addConstr(i + l <= 400000, name='Intermediate Luxury')
m.addConstr(t + v <= 275000, name='Truck Van')
m.addConstr( s + c -i -l >= 0, name='cars')

# average fuel economy constraint
m.addConstr(13*s + 7*c - 12*i - 15*l - 7*t - 2*v >= 0, name='mileage')

# market potential constraints
m.addConstr(s <= 600000, name='Subcompact market')
m.addConstr(c <= 400000, name='Compact market')
m.addConstr(i <= 300000, name='Intermediate market')
m.addConstr(l <= 225000, name='Luxury market')
m.addConstr(t <= 325000, name='Truck market')
m.addConstr(v <= 100000, name='Van market')

# implicit constraints
m.addConstr(s >=0, name='Zero_Subcompact')
m.addConstr(c >=0, name='Zero_Compact')
m.addConstr(i >=0, name='Zero_Intermediate')
m.addConstr(l >=0, name='Zero_Luxury')
m.addConstr(t >=0, name='Zero_Trucks')
m.addConstr(v >=0, name='Zero_Van')

# Optimize
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 18 rows, 6 columns and 34 nonzeros
Model fingerprint: 0x8fc6c118
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+02, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+06]
Presolve removed 12 rows and 0 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+08   2.050000e+06   0.000000e+00      0s
       6    3.5800000e+08   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds
Optimal objective  3.580000000e+08


In [4]:
# Convergence status
# we want 2, which means model was solved to optimality (subject to tolerances); optimal solution is available
print('Convergence Status:', m.status)

# Print results for each constraint
for v in m.getVars():
    print('%s: %g' % (v.varName, v.x))
    
# Print result for Objective using optimized constraints
print('Obj: %g' % m.objVal)

Convergence Status: 2
Subcompact: 320000
Compact: 300000
intermediate: 80000
Luxury: 225000
Truck: 275000
Van: 0
Obj: 3.58e+08


`Binding constraints` are when the right-hand side of the constraint equals the left-hand side. In other words, the value of the constraint when the solution values are used equals the original value of the parameter set by the problem.

In [5]:
# list to store binding vars
binding = []

# see slack values for vars
for c in m.getConstrs():
    print('%s: %g' % (c.ConstrName, c.slack))
    if c.slack==0:
        binding.append(c.ConstrName)

binding

Annual capacity: 0
subcompact compact: 0
Intermediate Luxury: 95000
Truck Van: 0
cars: -315000
mileage: 0
Subcompact market: 280000
Compact market: 100000
Intermediate market: 220000
Luxury market: 0
Truck market: 50000
Van market: 100000
Zero_Subcompact: -320000
Zero_Compact: -300000
Zero_Intermediate: -80000
Zero_Luxury: -225000
Zero_Trucks: -275000
Zero_Van: -0


['Annual capacity',
 'subcompact compact',
 'Truck Van',
 'mileage',
 'Luxury market',
 'Zero_Van']

<b>NOTE:</b> if the dual (shadow) price for a constraint is 0, then the constraint is NOT binding.

In [6]:
# Get shadow prices
m.printAttr('Pi')


  Constraint           Pi 
-------------------------
Annual capacity          100 
subcompact compact        212.5 
   Truck Van        212.5 
     mileage        -12.5 
Luxury market        212.5 


`Reduced cost` is the amount by which an objective function coefficient would have to improve before it would be possible for a corresponding variable to assume a positive value in the optimal solution.

In [7]:
# Ger reduce cost
m.printAttr('Rc')


    Variable           Rc 
-------------------------
         Van       -137.5 
